In [9]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import holidays
import pyodbc
from datetime import datetime

# Conectar a SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=192.168.192.21;'
                      'DATABASE=Auris_Personas;'
                      'UID=iazuaz;'
                      'PWD=192776775')

# Cargar datos desde la base de datos
query = "SELECT COD_SUC, FECHA, CANAL, HORA, ACEPTA_OFERTA FROM TB_SALTA_OFERTA_SALON_DIARIO_MODELO_PY"
df = pd.read_sql(query, conn)

# Preprocesamiento de la columna 'FECHA' y 'HORA'
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d-%m-%Y')
df['HORA'] = df['HORA'].astype(int)

# Crear las características temporales antes de hacer el groupby
df['DIA_LABORAL'] = df['FECHA'].dt.weekday < 5
df['FIN_DE_SEMANA'] = df['FECHA'].dt.weekday >= 5
df['DIA_SEMANA'] = df['FECHA'].dt.weekday
df['DIA_DEL_MES'] = df['FECHA'].dt.day
df['HORA_DEL_DIA'] = df['HORA']

# Agregar feriados chilenos
chile_holidays = holidays.Chile(years=[2023, 2024])
df['ES_FERIADO'] = df['FECHA'].dt.date.isin(chile_holidays.keys())

# Crear variables de lag y rolling para visitas previas (si aplicable)
df['LAG_VISITAS'] = df['ACEPTA_OFERTA'].shift(1)
df['ROLLING_VISITAS'] = df['ACEPTA_OFERTA'].rolling(window=3).mean()

# Eliminar filas con valores faltantes
df.dropna(inplace=True)

# Agrupar los datos por sucursal y por hora para obtener el conteo de ACEPTA_OFERTA
df_grouped = df.groupby(['COD_SUC', 'HORA']).agg({'ACEPTA_OFERTA': 'sum'}).reset_index()

# Después del groupby, las características temporales ya se calculan para cada fila
# Agregar las características temporales al dataframe agrupado
df_grouped = df_grouped.merge(df[['COD_SUC', 'HORA', 'DIA_LABORAL', 'FIN_DE_SEMANA', 'DIA_SEMANA', 'DIA_DEL_MES', 'HORA_DEL_DIA', 'ES_FERIADO', 'LAG_VISITAS', 'ROLLING_VISITAS']],
                              on=['COD_SUC', 'HORA'], how='left')

# Eliminar columnas redundantes
df_grouped = df_grouped[['COD_SUC', 'HORA', 'ACEPTA_OFERTA', 'DIA_LABORAL', 'FIN_DE_SEMANA', 'DIA_SEMANA', 'DIA_DEL_MES', 'HORA_DEL_DIA', 'ES_FERIADO', 'LAG_VISITAS', 'ROLLING_VISITAS']]

# Definir las características (X) y la variable objetivo (y)
X = df_grouped[['COD_SUC', 'DIA_LABORAL', 'FIN_DE_SEMANA', 'DIA_SEMANA', 'DIA_DEL_MES', 'HORA_DEL_DIA', 'ES_FERIADO', 'LAG_VISITAS', 'ROLLING_VISITAS']]
y = df_grouped['ACEPTA_OFERTA']

# Codificar la columna 'COD_SUC' (si es necesario)
X = pd.get_dummies(X, columns=['COD_SUC'], drop_first=True)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos a evaluar
models = {
    'XGBoost': XGBRegressor(eval_metric='mlogloss'),
    'LightGBM': LGBMRegressor(),
    'CatBoost': CatBoostRegressor(silent=True)
}

# Evaluar el rendimiento de cada modelo
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MAE': mae, 'R²': r2}

# Mostrar los resultados
print(pd.DataFrame(results).T)


C:\Users\iazuaz\AppData\Local\Temp\ipykernel_22904\3839249845.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018587 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 224
[LightGBM] [Info] Number of data points in the train set: 1123061, number of used features: 85
[LightGBM] [Info] Start training from score 361.378474
                MAE        R²
XGBoost    9.030123  0.990922
LightGBM  14.390441  0.987572
CatBoost   4.179750  0.994272


In [14]:
import holidays
cl_holidays = holidays.Chile()
for ptr in holidays.Chile(years = 2025).items():
    print(ptr)

(datetime.date(2025, 1, 1), "New Year's Day")
(datetime.date(2025, 4, 18), 'Good Friday')
(datetime.date(2025, 4, 19), 'Holy Saturday')
(datetime.date(2025, 5, 1), 'Labor Day')
(datetime.date(2025, 5, 21), 'Navy Day')
(datetime.date(2025, 6, 20), 'National Day of Indigenous Peoples')
(datetime.date(2025, 6, 29), "Saint Peter and Saint Paul's Day")
(datetime.date(2025, 7, 16), 'Our Lady of Mount Carmel')
(datetime.date(2025, 8, 15), 'Assumption Day')
(datetime.date(2025, 9, 18), 'Independence Day')
(datetime.date(2025, 9, 19), 'Army Day')
(datetime.date(2025, 10, 12), "Meeting of Two Worlds' Day")
(datetime.date(2025, 10, 31), 'Reformation Day')
(datetime.date(2025, 11, 1), "All Saints' Day")
(datetime.date(2025, 12, 8), 'Immaculate Conception')
(datetime.date(2025, 12, 25), 'Christmas Day')
